# Land To Bronze

This notebook sets up the DLT pipeline from the Landing volume to the bronze layer

In [0]:
import dlt
from pyspark.sql import functions as F

source = spark.conf.get("source")


In [0]:
@dlt.view(
  name = "raw_data",
  comment="Test_data"
)
def data_raw():
  df = (
    spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "csv")
      .option("cloudFiles.schemaLocation", f"{source}/landing/inferred_schema")
      .option("rescuedDataColumn", "_rescued_data")
      .option("header","true")
      .option("cloudFiles.inferColumnTypes", "true")
      .option("cloudFiles.schemaHints", "timestamp timestamp")
      .load(source +"/")
  )
  return df.withColumn("filename", F.col("_metadata.file_path"))
  


@dlt.table(
    name = "bronze_data_test",
    comment="New readings - exclude rows with dates in the future",
    table_properties = {
      "quality": "bronze"
      }
)
def data_bronze():
  df = dlt.read_stream("raw_data")
  return df.withColumn("processedtime",F.current_timestamp())